In [85]:
import tensorflow.compat.v1 as tf
from tensorflow import feature_column
from tensorflow.keras import layers
tf.disable_v2_behavior()
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb

# from tensorflow import keras
# from tensorflow.keras.models import Sequential # 케라스의 Sequential()을 임포트
# from tensorflow.keras.layers import Dense # 케라스의 Dense()를 임포트
# from tensorflow.keras import optimizers # 케라스의 옵티마이저를 임포트
%matplotlib inline

In [86]:
seed = 0
# a = tf.Variable(tf.random_normal([1], dtype=tf.float64, seed=0))
# b = tf.Variable(tf.random_normal([1], dtype=tf.float64, seed=0))

# tensorflow, numpy 랜덤 값을 설정합니다.
tf.set_random_seed(seed)
np.random.seed(seed)

In [120]:
#doginfo.csv파일 데이터를 pandas를 이용해 읽어옵니다.
dog_data=pd.read_csv("doginfo.csv")
dog_train=pd.read_csv("testDog.csv")
kindCd=pd.read_csv("kindCd.csv")

dog_data = dog_data.dropna(axis=0)
dog_train = dog_train.dropna(axis=0)
kindCd_data = kindCd.dropna(axis=0)

dog_data
# print(dog_train)
# print(kindCd)

,kindNum,neuterYn,sexCd,weight,noticeDays,age2,processState
0,128.0,N,F,7.46,10,12,0
1,114.0,N,M,7.00,14,1,1
2,114.0,U,M,4.50,11,2,0
3,67.0,N,M,10.00,8,1,0
4,114.0,N,M,6.00,8,4,0
...,...,...,...,...,...,...,...
22787,114.0,N,M,1.00,10,0,1
22788,114.0,N,M,1.00,10,0,1
22789,114.0,N,M,1.00,10,0,1
22790,128.0,U,F,6.00,12,3,0


In [121]:
kindCd = np.array(kindCd_data, dtype = np.float64)
kindCd_train=np.array(kindCd_data, dtype=np.float64)

kindCd = kindCd.reshape(177)
kindCd_train=kindCd_train.reshape(177)
print(kindCd)
print(kindCd.shape)

[ 54.  56.  55. 118. 115.  37.  81. 204.  83.  82.  38.  39.  40.  43.
  42. 153.  41. 120. 155.  69.  71. 142.  93. 167.  70. 166.  94. 121.
 152.  73. 146.  72. 159.  76.  75.  79.  78.  77.  74.  80. 114. 133.
  12.  17.  15. 164. 157. 148.  16.  20.  21.  22.  24. 208.  23.  26.
  27. 169.  25.  19.  13.  18.  14. 162.  85.  96.  95.   1.  34. 104.
  31.  99. 122. 123.  97. 132. 105. 154. 124. 100. 103. 151. 139. 101.
 102.  98. 136. 202. 160. 203.   8. 131.   9. 119. 150. 210.  57.  58.
  59.   6.   4.   7.   5. 143.  11.  10. 137.  84. 163. 112. 113. 149.
 211. 110. 205. 108. 109.  60.  46.  47.  44.  45.  53.  62.  61.  52.
 165.  51. 156. 129.  67.  35.  33.  32. 158. 144.  30.  29.  64. 207.
  28.   2.  68. 125. 141. 145.  36.  66.  65.  63. 140. 107. 106. 209.
  86.  88.  90.  87. 138.  89. 126. 127. 128.  91.   3. 161.  50. 168.
  49. 147.  92.  48. 135. 206. 130. 134. 111.]
(177,)


In [122]:
# kindNum을 원핫 인코딩
kindCd = pd.concat((pd.get_dummies(dog_data.kindNum, columns=kindCd), pd.DataFrame(columns=kindCd))).fillna(0)
# kindCd

# 학습데이터에서 kindNum 열을 삭제한 후, 원핫 인코딩된 kindCd를 붙임
dog_data.drop(['kindNum'], axis='columns', inplace=True)
dog_data = pd.concat([dog_data, kindCd], axis=1)


# kindNum을 원핫 인코딩
kindCd_train = pd.concat((pd.get_dummies(dog_train.kindNum, columns=kindCd_train), pd.DataFrame(columns=kindCd_train))).fillna(0)

# 테스트데이터에서 kindNum 열을 삭제한 후, 원핫 인코딩된 kindCd를 붙임
dog_train.drop(['kindNum'], axis='columns', inplace=True)
dog_train = pd.concat([dog_train, kindCd_train], axis=1)
dog_data
# dog_train

,neuterYn,sexCd,weight,noticeDays,age2,processState,1.0,2.0,3.0,4.0,...,202.0,203.0,204.0,205.0,206.0,207.0,208.0,209.0,210.0,211.0
0,N,F,7.46,10,12,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,N,M,7.00,14,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,U,M,4.50,11,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,N,M,10.00,8,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,N,M,6.00,8,4,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22787,N,M,1.00,10,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
22788,N,M,1.00,10,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
22789,N,M,1.00,10,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
22790,U,F,6.00,12,3,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [123]:
def dummy_data(data, columns):
    for column in columns:
        data = pd.concat([data, pd.get_dummies(data[column], prefix = column)], axis = 1)
        data = data.drop(column, axis = 1)
    return data

In [124]:
dummy_columns = ["neuterYn", "sexCd"]
data = dummy_data(dog_data, dummy_columns)
train_data = dummy_data(dog_train, dummy_columns)
data.head()
train_data.head()
# print(type(dog_data))
# print(type(data))

print(data)

data = np.array(data, dtype = np.float64)
train_data = np.array(train_data, dtype = np.float64)

# data
# print(data.shape)
# print(len(train_data))
# print(train_data.shape)

       weight  noticeDays  age2  processState  1.0  2.0  3.0  4.0  5.0  6.0  \
0        7.46          10    12             0    0    0    0    0    0    0   
1        7.00          14     1             1    0    0    0    0    0    0   
2        4.50          11     2             0    0    0    0    0    0    0   
3       10.00           8     1             0    0    0    0    0    0    0   
4        6.00           8     4             0    0    0    0    0    0    0   
...       ...         ...   ...           ...  ...  ...  ...  ...  ...  ...   
22787    1.00          10     0             1    0    0    0    0    0    0   
22788    1.00          10     0             1    0    0    0    0    0    0   
22789    1.00          10     0             1    0    0    0    0    0    0   
22790    6.00          12     3             0    0    0    0    0    0    0   
22791    3.50          10     0             0    0    0    0    0    0    0   

       ...  208.0  209.0  210.0  211.0  neuterYn_N 

In [125]:
a = data[:, :3]
b = data[:, 4:]

# print(a[0,:])
# print(b)

#numpy 배열에서 데이터 변화요인(kindCd, neuterYn, sexCd, weight, noticeDays, age2)으로 사용할 데이터를 뽑아냅니다.
xData = np.concatenate([a, b], axis = 1)

a = train_data[:, :3]
b = train_data[:, 4:]
testX = np.concatenate([a, b], axis = 1)

print(xData)
type(xData)
xData.shape
testX.shape

[[ 7.46 10.   12.   ...  1.    0.    0.  ]
 [ 7.   14.    1.   ...  0.    1.    0.  ]
 [ 4.5  11.    2.   ...  0.    1.    0.  ]
 ...
 [ 1.   10.    0.   ...  0.    1.    0.  ]
 [ 6.   12.    3.   ...  1.    0.    0.  ]
 [ 3.5  10.    0.   ...  0.    1.    0.  ]]


(610, 186)

In [126]:
#numpy배열에서 결과(입양여부)로 사용할 데이터를 뽑아냅니다.
yData=data[:,[3]]
testY = train_data[:,[3]]

# yData = yData.astype('int32')
print(yData)
type(yData)
yData.dtype

[[0.]
 [1.]
 [0.]
 ...
 [1.]
 [0.]
 [0.]]


dtype('float64')

In [127]:
# model=Sequential()
# model.add(Dense(1, input_dim=186, activation='sigmoid'))
# sgd=optimizers.SGD(lr=0.01)
# model.compile(optimizer=sgd ,loss='binary_crossentropy',metrics=['binary_accuracy'])
# # 옵티마이저는 경사하강법 sgd를 사용합니다.
# # 손실 함수(Loss function)는 binary_crossentropy(이진 크로스 엔트로피)를 사용합니다.
# model.fit(xData,yData, batch_size=128, epochs=200, shuffle=False)
# # 주어진 X와 y데이터에 대해서 오차를 최소화하는 작업을 200번 시도합니다.

In [133]:
# l2_model = keras.models.Sequential([
#     keras.layers.Dense(16, kernel_regularizer=keras.regularizers.l2(0.001),
#                        activation=tf.nn.relu, input_shape=(186,)),
#     keras.layers.Dense(16, kernel_regularizer=keras.regularizers.l2(0.001),
#                        activation=tf.nn.relu),
#     keras.layers.Dense(1, activation=tf.nn.sigmoid)
# ])

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, BatchNormalization, Dropout
from tensorflow.keras.optimizers import Adadelta


def mlp_model():
    model = Sequential()

#     model = Sequential([
#     Dense(16, kernel_regularizer=keras.regularizers.l2(0.001),
#                        activation=tf.nn.relu, input_shape=(186,)),
#     Dense(16, kernel_regularizer=keras.regularizers.l2(0.001),
#                        activation=tf.nn.relu),
#     Dense(1, activation=tf.nn.sigmoid)
# ])

    model.add(Dense(16, input_shape = (186, )))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(Dense(32))
    model.add(Activation('relu'))
#     model.add(BatchNormalization())
#     model.add(Dense(32))
#     model.add(Activation('relu'))
#     model.add(Dense(50, input_shape = (186, )))
#     model.add(BatchNormalization())                    # Add Batchnorm layer before Activation
#     model.add(Activation('sigmoid'))    
#     model.add(Dense(50))
#     model.add(BatchNormalization())                    # Add Batchnorm layer before Activation
#     model.add(Activation('sigmoid'))    
#     model.add(Dense(50))
#     model.add(BatchNormalization())                    # Add Batchnorm layer before Activation
#     model.add(Activation('sigmoid'))    
#     model.add(Dense(50))
    model.add(BatchNormalization())                    # Add Batchnorm layer before Activation
    model.add(Dense(1))
    model.add(Activation('sigmoid'))   
#     model.add(Dropout(0.5))
#     model.add(Dense(1))
#     model.add(Activation('softmax'))

#     sgd=optimizers.SGD(lr=0.01, momentum=0.9)

    model.compile(optimizer=Adadelta(rho=0.95),
                     loss='binary_crossentropy',
                     metrics=['accuracy', 'binary_crossentropy'])
    return model

# mlp_model(l2_model)

# sgd=optimizers.SGD(lr=0.01)

# l2_model.compile(optimizer='sgd',
#                  loss='binary_crossentropy',
#                  metrics=['accuracy', 'binary_crossentropy'])

l2_model = mlp_model()

l2_model_history = l2_model.fit(xData, yData,
                                epochs=100,
                                shuffle = True,
                                batch_size=32,
                                validation_data=(testX, testY),
                                verbose=1)

l2_model.summary()

loss_and_metric = l2_model.evaluate(testX, testY, batch_size = 32, verbose = 0)
print("train, loss and metric : {}".format(loss_and_metric))


Train on 22777 samples, validate on 610 samples
Epoch 1/100
22777/22777 [==============================] - 3s 141us/sample - loss: 0.9175 - acc: 0.4521 - binary_crossentropy: 0.9175 - val_loss: 0.9292 - val_acc: 0.4672 - val_binary_crossentropy: 0.9292
Epoch 2/100
22777/22777 [==============================] - 3s 118us/sample - loss: 0.9040 - acc: 0.4554 - binary_crossentropy: 0.9040 - val_loss: 0.9202 - val_acc: 0.4672 - val_binary_crossentropy: 0.9202
Epoch 3/100
22777/22777 [==============================] - 4s 172us/sample - loss: 0.8920 - acc: 0.4569 - binary_crossentropy: 0.8920 - val_loss: 0.9077 - val_acc: 0.4639 - val_binary_crossentropy: 0.9077
Epoch 4/100
22777/22777 [==============================] - 5s 204us/sample - loss: 0.8811 - acc: 0.4571 - binary_crossentropy: 0.8811 - val_loss: 0.8984 - val_acc: 0.4705 - val_binary_crossentropy: 0.8984
Epoch 5/100
22777/22777 [==============================] - 3s 139us/sample - loss: 0.8728 - acc: 0.4597 - binary_crossentropy: 0.872

22777/22777 [==============================] - 3s 124us/sample - loss: 0.7215 - acc: 0.5261 - binary_crossentropy: 0.7215 - val_loss: 0.7264 - val_acc: 0.5197 - val_binary_crossentropy: 0.7264
Epoch 41/100
22777/22777 [==============================] - 3s 147us/sample - loss: 0.7170 - acc: 0.5303 - binary_crossentropy: 0.7170 - val_loss: 0.7288 - val_acc: 0.5213 - val_binary_crossentropy: 0.7288
Epoch 42/100
22777/22777 [==============================] - 5s 217us/sample - loss: 0.7160 - acc: 0.5312 - binary_crossentropy: 0.7160 - val_loss: 0.7296 - val_acc: 0.5295 - val_binary_crossentropy: 0.7296
Epoch 43/100
22777/22777 [==============================] - 4s 178us/sample - loss: 0.7153 - acc: 0.5332 - binary_crossentropy: 0.7153 - val_loss: 0.7257 - val_acc: 0.5279 - val_binary_crossentropy: 0.7257
Epoch 44/100
22777/22777 [==============================] - 4s 155us/sample - loss: 0.7127 - acc: 0.5352 - binary_crossentropy: 0.7127 - val_loss: 0.7254 - val_acc: 0.5279 - val_binary_cros

Epoch 80/100
22777/22777 [==============================] - 3s 120us/sample - loss: 0.6832 - acc: 0.5751 - binary_crossentropy: 0.6832 - val_loss: 0.6977 - val_acc: 0.5475 - val_binary_crossentropy: 0.6977
Epoch 81/100
22777/22777 [==============================] - 3s 125us/sample - loss: 0.6831 - acc: 0.5753 - binary_crossentropy: 0.6831 - val_loss: 0.6967 - val_acc: 0.5475 - val_binary_crossentropy: 0.6967
Epoch 82/100
22777/22777 [==============================] - 3s 144us/sample - loss: 0.6839 - acc: 0.5721 - binary_crossentropy: 0.6839 - val_loss: 0.6974 - val_acc: 0.5508 - val_binary_crossentropy: 0.6974
Epoch 83/100
22777/22777 [==============================] - 4s 196us/sample - loss: 0.6805 - acc: 0.5779 - binary_crossentropy: 0.6805 - val_loss: 0.6958 - val_acc: 0.5475 - val_binary_crossentropy: 0.6958
Epoch 84/100
22777/22777 [==============================] - 5s 212us/sample - loss: 0.6806 - acc: 0.5760 - binary_crossentropy: 0.6806 - val_loss: 0.6972 - val_acc: 0.5525 - va

In [130]:
# for i in range(len(train_data)):
#     new_x=testX[i, :].reshape(1,186)
#     print('입양 확률 : %8.4f%%' % (l2_model.predict(new_x)*100))
l2_model.save('l2_model_yj')